# Rearc Data Quest - Part 3

In [9]:

# --- Configuration -----------------------------------------------------------
USE_S3 = True
AWS_REGION = "us-east-2"
AWS_PROFILE = None

S3_BUCKET = "rearc-dataquest-harpreet"
S3_KEY_BLS = "part1/bls/pr/pr.data.0.Current"
S3_KEY_POP = "part2/population_data.json"

LOCAL_BLS_CSV = "./data/pr.data.0.Current"
LOCAL_POP_JSON = "./data/population_data.json"

OUTDIR_LOCAL = "./outputs"
S3_PREFIX_OUT = "part3/outputs"
# ----------------------------------------------------------------------------


In [ ]:

# %pip install -q pandas boto3 requests


In [10]:

import json, io
from pathlib import Path
import pandas as pd

def _boto3_client(region: str, profile: str|None=None):
    import boto3
    if profile:
        from botocore.session import Session
        sess = Session(profile=profile)
        return sess.create_client("s3", region_name=region)
    return boto3.client("s3", region_name=region)

def _ensure_outdir(path: str):
    Path(path).mkdir(parents=True, exist_ok=True)

def s3_read_bytes(bucket: str, key: str, region: str, profile: str|None=None) -> bytes:
    key = key.lstrip("/")
    s3 = _boto3_client(region, profile)
    obj = s3.get_object(Bucket=bucket, Key=key)
    return obj["Body"].read()

def read_population_df(local_path: str, bucket: str=None, key: str=None, use_s3: bool=False, region: str=None, profile: str|None=None) -> pd.DataFrame:
    if use_s3:
        raw = s3_read_bytes(bucket, key, region, profile).decode("utf-8")
    else:
        raw = Path(local_path).read_text(encoding="utf-8")
    parsed = json.loads(raw)
    if isinstance(parsed, dict) and "data" in parsed:
        df = pd.DataFrame(parsed["data"])
    else:
        df = pd.DataFrame(parsed)
    rename_map = {}
    for c in df.columns:
        cl = c.lower()
        if cl == "year": rename_map[c] = "year"
        if cl == "population": rename_map[c] = "Population"
    df = df.rename(columns=rename_map)
    keep = [c for c in df.columns if c in ("year","Population")]
    if not keep:
        raise ValueError(f"Population JSON lacks expected fields. Columns: {list(df.columns)}")
    df = df[keep].copy()
    df["year"] = pd.to_numeric(df["year"], errors="coerce").astype("Int64")
    df["Population"] = pd.to_numeric(df["Population"], errors="coerce")
    return df

def read_bls_df(local_path: str, bucket: str=None, key: str=None, use_s3: bool=False, region: str=None, profile: str|None=None) -> pd.DataFrame:
    if use_s3:
        raw_bytes = s3_read_bytes(bucket, key, region, profile)
        try:
            df = pd.read_csv(io.BytesIO(raw_bytes), sep=r"\s+", engine="python")
        except Exception:
            df = pd.read_fwf(io.BytesIO(raw_bytes))
        if df.shape[1] == 1:
            df = pd.read_fwf(io.BytesIO(raw_bytes))
    else:
        p = Path(local_path)
        if not p.exists():
            raise FileNotFoundError(f"Local BLS file missing: {p.resolve()}")
        try:
            df = pd.read_csv(p, sep=r"\s+", engine="python")
        except Exception:
            df = pd.read_fwf(p)

    df.columns = [str(c).strip().lower() for c in df.columns]
    required = {"series_id","year","period","value"}
    missing = required - set(df.columns)
    if missing:
        raise AssertionError(f"BLS file missing columns: {missing}. Present: {list(df.columns)}")
    df["series_id"] = df["series_id"].astype(str).str.strip()
    df["period"] = df["period"].astype(str).str.strip().str.upper()
    df["year"] = pd.to_numeric(df["year"], errors="coerce").astype("Int64")
    df["value"] = pd.to_numeric(df["value"], errors="coerce")
    return df

def write_output_csv(df: pd.DataFrame, local_path: str, bucket: str=None, key: str=None, use_s3: bool=False, region: str=None, profile: str|None=None):
    _ensure_outdir(Path(local_path).parent)
    df.to_csv(local_path, index=False)
    if use_s3:
        data = Path(local_path).read_bytes()
        s3 = _boto3_client(region, profile)
        s3.put_object(Bucket=bucket, Key=key.lstrip("/"), Body=data, ContentType="text/csv")
    return local_path


### Load datasets

In [11]:

bls = read_bls_df(
    local_path=LOCAL_BLS_CSV, bucket=S3_BUCKET, key=S3_KEY_BLS, use_s3=USE_S3, region=AWS_REGION, profile=AWS_PROFILE
)
pop = read_population_df(
    local_path=LOCAL_POP_JSON, bucket=S3_BUCKET, key=S3_KEY_POP, use_s3=USE_S3, region=AWS_REGION, profile=AWS_PROFILE
)

print("BLS shape:", bls.shape)
print("Population shape:", pop.shape)

bls_q = bls[bls["period"].str.match(r"^Q0[1-4]$", na=False)].copy()
bls_q.head(3)


BLS shape: (37239, 5)
Population shape: (10, 2)


,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.6,None
1,PRS30006011,1995,Q02,2.1,None
2,PRS30006011,1995,Q03,0.9,None


## Task 1 - Mean/std computation

In [12]:

# Compute on the Series and build a one-row DataFrame
pop_2013_2018 = pop.query("year >= 2013 and year <= 2018").dropna(subset=["Population"]).copy()

summary_series = pop_2013_2018["Population"].agg(["mean", "std"]).round(2)
summary_pop = summary_series.rename({
    "mean": "mean_population",
    "std": "std_population"
}).to_frame().T

summary_pop


,mean_population,std_population
Population,322069808.0,4158441.04


## Task 2 - Best year per series

In [13]:

series_year_sums = (
    bls_q.groupby(["series_id","year"], dropna=False)["value"]
        .sum(min_count=1)
        .reset_index(name="year_sum")
)
best_rows = (
    series_year_sums.sort_values(["series_id","year"])
    .loc[series_year_sums.groupby("series_id")["year_sum"].idxmax()]
    .reset_index(drop=True)
)
best_year_per_series = best_rows.rename(columns={"year_sum":"value"})
best_year_per_series.head(10)


,series_id,year,value
0,PRS30006011,2022,16.400
1,PRS30006012,2022,13.000
2,PRS30006013,1998,564.713
3,PRS30006021,2010,14.200
4,PRS30006022,2010,8.900
5,PRS30006023,2014,402.512
6,PRS30006031,2022,16.400
7,PRS30006032,2021,13.900
8,PRS30006033,1998,561.703
9,PRS30006061,2022,29.600


## Task 3 - PRS30006032 Q01 with population

In [14]:

target_series = "PRS30006032"
target_period = "Q01"

subset = (
    bls_q.query("series_id == @target_series and period == @target_period")
        .loc[:, ["series_id","year","period","value"]]
        .copy()
        .sort_values(["year"])
)
joined = subset.merge(pop, on="year", how="left")
joined.head(12)


,series_id,year,period,value,Population
0,PRS30006032,1995,Q01,0.0,NaN
1,PRS30006032,1996,Q01,-4.2,NaN
2,PRS30006032,1997,Q01,2.8,NaN
3,PRS30006032,1998,Q01,0.9,NaN
4,PRS30006032,1999,Q01,-4.1,NaN
5,PRS30006032,2000,Q01,0.5,NaN
6,PRS30006032,2001,Q01,-6.3,NaN
7,PRS30006032,2002,Q01,-6.6,NaN
8,PRS30006032,2003,Q01,-5.7,NaN
9,PRS30006032,2004,Q01,2.0,NaN


### Save outputs

In [15]:

out1 = write_output_csv(summary_pop, f"{OUTDIR_LOCAL}/population_mean_std_2013_2018.csv",
                        bucket=S3_BUCKET, key=f"{S3_PREFIX_OUT}/population_mean_std_2013_2018.csv",
                        use_s3=USE_S3, region=AWS_REGION, profile=AWS_PROFILE)
out2 = write_output_csv(best_year_per_series, f"{OUTDIR_LOCAL}/best_year_per_series.csv",
                        bucket=S3_BUCKET, key=f"{S3_PREFIX_OUT}/best_year_per_series.csv",
                        use_s3=USE_S3, region=AWS_REGION, profile=AWS_PROFILE)
out3 = write_output_csv(joined, f"{OUTDIR_LOCAL}/prs30006032_q01_with_population.csv",
                        bucket=S3_BUCKET, key=f"{S3_PREFIX_OUT}/prs30006032_q01_with_population.csv",
                        use_s3=USE_S3, region=AWS_REGION, profile=AWS_PROFILE)

print("Wrote:", out1)
print("Wrote:", out2)
print("Wrote:", out3)


Wrote: ./outputs/population_mean_std_2013_2018.csv
Wrote: ./outputs/best_year_per_series.csv
Wrote: ./outputs/prs30006032_q01_with_population.csv
